In [1]:
import pandas as pd

import urllib.request, urllib.parse, urllib.error
import os, json, requests
import json
import ssl

df = pd.read_csv('./Dados/cidades_regiao_cafe.csv', encoding ='latin1')
df

,Região,Município,UF
0,Sul de Minas,Aiuruoca,MG
1,Sul de Minas,Tapiraí,MG
2,Sul de Minas,Cabo Verde,MG
3,Sul de Minas,Contagem,MG
4,Sul de Minas,Estiva,MG
...,...,...,...
1416,Mato Grosso,Paranaíta,MT
1417,Mato Grosso,Rondolândia,MT
1418,Mato Grosso,Santo Antônio do Leverger,MT
1419,Mato Grosso,Tangará da Serra,MT


In [2]:
def get_estado(uf):
    estados = {
        'AC': 'Acre',
        'AL':'Alagoas',
        'AP':'Amapá',
        'AM':'Amazonas',
        'BA':'Bahia',
        'CE':'Ceará',
        'ES':'Espírito Santo',
        'GO':'Goiás',
        'MA':'Maranhão',
        'MT':'Mato Grosso',
        'MS':'Mato Grosso do Sul',
        'MG':'Minas Gerais',
        'PA':'Pará',
        'PB':'Paraíba',
        'PR':'Paraná',
        'PE': 'Pernambuco',
        'PI':'Piauí',
        'RJ':'Rio de Janeiro',
        'RN':'Rio Grande do Norte',
        'RS':'Rio Grande do Sul',
        'RO':'Rondônia',
        'RR':'Roraima',
        'SC':'Santa Catarina',
        'SP':'São Paulo',
        'SE':'Sergipe',
        'TO': 'Tocantins',
        'DF': 'Distrito Federal'
            }
    return estados[uf]

df['Estado'] = df.apply(lambda x: get_estado(x.loc['UF']), axis=1).copy()
df

,Região,Município,UF,Estado
0,Sul de Minas,Aiuruoca,MG,Minas Gerais
1,Sul de Minas,Tapiraí,MG,Minas Gerais
2,Sul de Minas,Cabo Verde,MG,Minas Gerais
3,Sul de Minas,Contagem,MG,Minas Gerais
4,Sul de Minas,Estiva,MG,Minas Gerais
...,...,...,...,...
1416,Mato Grosso,Paranaíta,MT,Mato Grosso
1417,Mato Grosso,Rondolândia,MT,Mato Grosso
1418,Mato Grosso,Santo Antônio do Leverger,MT,Mato Grosso
1419,Mato Grosso,Tangará da Serra,MT,Mato Grosso


In [3]:
df['location'] = df['Município'] + ", state of " + df['Estado'] + ", Brazil"
df['location'].head(10)

0         Aiuruoca, state of Minas Gerais, Brazil
1          Tapiraí, state of Minas Gerais, Brazil
2       Cabo Verde, state of Minas Gerais, Brazil
3         Contagem, state of Minas Gerais, Brazil
4           Estiva, state of Minas Gerais, Brazil
5         Itaguara, state of Minas Gerais, Brazil
6        Liberdade, state of Minas Gerais, Brazil
7     Passa Quatro, state of Minas Gerais, Brazil
8        Rio Manso, state of Minas Gerais, Brazil
9    Sapucaí-Mirim, state of Minas Gerais, Brazil
Name: location, dtype: object

In [4]:
'''
This cell is mostly from:
https://www.codingforentrepreneurs.com/blog/python-tutorial-google-geocoding-api
'''
    GOOGLE_API_KEY = 'put your api key here' 

import time
def get_gps(address_or_zipcode):
    time.sleep(0.1)
    lat, lng = None, None
    api_key = GOOGLE_API_KEY
    base_url = "https://maps.googleapis.com/maps/api/geocode/json"
    endpoint = f"{base_url}?address={address_or_zipcode}&key={api_key}"
    # see how our endpoint includes our API key? Yes this is yet another reason to restrict the key
    r = requests.get(endpoint)
    if r.status_code not in range(200, 299):
        print('None')
        return None
    try:
        '''
        This try block incase any of our inputs are invalid. This is done instead
        of actually writing out handlers for all kinds of responses.
        '''
        results = r.json()['results'][0]
        lat = results['geometry']['location']['lat']
        lng = results['geometry']['location']['lng']
        city = results['address_components'][0]['long_name']
        ready = str(lat)+','+str(lng)+','+city
        print(ready)
        return ready
    except:
        print('None')
        return None

In [5]:
%%time
#it takes hours to run
df['(Lat, Long, City)'] = df.apply(lambda x: get_gps(x.loc['location']), axis=1)
df

-21.9750633,-44.6036109,Aiuruoca
-19.8868478,-46.0200063,Tapiraí
-21.4730317,-46.3970615,Cabo Verde
-19.9384589,-44.0518344,Contagem
-22.465012,-46.0157038,Estiva
-20.3922738,-44.486416,Itaguara
-22.0277329,-44.3200076,Liberdade
-22.3829371,-44.9666211,Passa Quatro
-20.266236,-44.3089514,Rio Manso
-22.7472478,-45.7431742,Sapucaí-Mirim
-20.9188537,-46.984882,São Sebastião do Paraíso
-20.9431969,-45.3915696,Aguanil
-21.3654678,-46.9418638,Arceburgo
-21.9234119,-46.3886147,Caldas
-21.1901619,-45.4393631,Coqueiral
-22.8542299,-46.3189034,Extrema
-22.4247429,-45.4601323,Itajubá
-21.5107696,-44.9041132,Luminárias
-20.6523602,-44.4954993,Passa Tempo
-21.1202699,-44.2246696,Santa Cruz de Minas
-20.0359196,-44.1456593,Sarzedo
-20.4092733,-44.0257154,Belo Vale
-22.171384,-44.6438278,Alagoa
-22.7563618,-46.1479233,Camanducaia
-21.7925148,-45.7000359,Cordislândia
-21.4073667,-45.8301095,Fama
-22.2830147,-44.8698779,Itamonte
-21.6803986,-45.919,Machado
-22.2086435,-44.2358985,Passa-Vinte
-22.028798

-16.7291552,-43.8670745,Montes Claros
-18.8282807,-42.4393551,Santa Efigênia de Minas
-18.3612056,-43.1708836,Serra Azul de Minas
-15.0829225,-43.1185339,Gameleiras
-16.3660734,-42.5146782,Padre Carvalho
-17.3978038,-42.7364016,Veredinha
-17.0834392,-40.933945,Águas Formosas
-15.9677317,-41.4959964,Cachoeira de Pajeú
-18.2441852,-43.598022,Diamantina
-17.0634045,-43.3082631,Itacambira
-17.8442329,-42.0789785,Malacacheta
-15.170387,-42.49871599999999,Montezuma
-16.9385841,-40.6821806,Santa Helena de Minas
-17.7886344,-40.2446234,Serra dos Aimorés
-16.8501057,-43.69667099999999,Glaucilândia
-17.0749651,-41.4853049,Padre Paraíso
-16.8044995,-42.3401051,Virgem da Lapa
-15.7463844,-41.46142280000001,Águas Vermelhas
-18.2387866,-41.7258273,Campanário
-18.7770317,-41.4792108,Divino das Laranjeiras
-17.4020879,-41.6700625,Itaipé
-15.0487812,-42.9459702,Mamonas
-18.4578824,-42.2491936,Nacip Raydan
-18.7246846,-41.1616361,São João do Manteninha
-15.8114454,-42.87097620000001,Serranópolis de Mina

-20.6882004,-42.6172434,Canaã
-20.6087581,-42.1450035,Divino
-20.2560957,-42.0334316,Manhuaçu
-19.8164299,-41.4376773,Mutum
-20.2435264,-41.9839846,Reduto
-20.3941537,-42.1515233,São João do Manhuaçu
-20.7561451,-42.8800278,Viçosa
-20.5219536,-42.3292384,Pedra Bonita
-20.1069834,-41.9259704,Santana do Manhuaçu
-20.8573353,-42.8013491,Coimbra
-20.1522059,-41.6232158,Lajinha
-20.17268,-42.2707395,Caputira
-20.2040323,-41.7958167,Durandé
-20.2848214,-42.3414891,Matipó
-20.506208,-42.2082033,Orizânia
-20.7061087,-42.71932779999999,São Miguel do Anta
-21.0700859,-42.6376269,São Sebastião da Vargem Alegre
-20.8655428,-42.2453312,Vieiras
-20.832037,-42.1553798,Pedra Dourada
-20.475341,-42.4831909,Sericita
-21.0989202,-42.18220609999999,Eugenópolis
-20.4387375,-42.1016354,Luisburgo
-20.3624576,-43.1436736,Acaiaca
-20.7934404,-43.9302198,Casa Grande
-21.1095356,-44.02127960000001,Dores de Campos
-20.6765494,-43.6099567,Itaverava
-20.4321474,-42.8046197,Oratórios
-20.2258234,-42.6515412,Rio Casc

-20.0584856,-48.1090128,Água Comprida
-19.6972963,-49.0836212,Comendador Gomes
-18.5896484,-49.1970562,Centralina
-19.762486,-48.5714802,Campo Florido
-19.9241693,-48.3811173,Conceição das Alagoas
-19.9719267,-47.7757685,Delta
-19.4614059,-45.5957533,Dores do Indaiá
-20.0246311,-45.540565,Lagoa da Prata
-17.8368328,-46.5212205,Lagoa Grande
-19.3079524,-48.9265496,Prata
-19.8603066,-49.7710053,São Francisco de Sales
-19.157438,-45.4447859,Abaeté
-18.6829575,-49.5706164,Capinópolis
-19.1464575,-45.7138281,Cedro do Abaeté
-17.0103224,-46.0085329,Brasilândia de Minas
-20.0263494,-48.9362703,Frutal
-19.5221638,-45.787951,Estrela do Indaiá
-19.9018142,-49.3701088,Itapagipe
-19.790283,-45.6820678,Luz
-20.1399952,-48.701023,Planura
-19.270648,-45.55732219999999,Quartel Geral
-18.4868648,-47.4030649,Abadia dos Dourados
-15.6288264,-46.4298542,Buritis
-18.5787112,-47.876108,Cascalho Rico
-18.4309993,-47.6081353,Douradoquara
-19.4878116,-46.5450499,Ibiá
-18.7776633,-46.405353,Lagoa Formosa
-19.99

-12.8065443,-41.3302644,Andaraí
-12.7099665,-41.82849090000001,Boninal
-13.4136556,-41.284438,Ibicoara
-13.2930206,-40.9597382,Iramaia
-12.9810717,-40.9654076,Itaeté
-12.5623255,-41.3886026,Lençóis
-11.5492584,-41.1572668,Morro do Chapéu
-12.8164871,-41.0720116,Nova Redenção
-12.5146538,-41.57816210000001,Palmeiras
-13.580844,-41.8134444,Rio de Contas
-12.0880843,-41.6478049,Souto Soares
-12.2826722,-41.1725415,Wagner
-14.6126249,-41.1380494,Anagé
-14.8659489,-40.5778538,Barra do Choça
-14.337029,-40.9094716,Caetanos
-14.7244929,-41.2598571,Caraíbas
-14.6825242,-42.5050773,Licínio de Almeida
-14.2389228,-40.7721729,Mirante
-14.943269,-41.7186194,Piripá
-15.4592728,-40.74063779999999,Ribeirão do Largo
-14.8619237,-40.8445346,Vitória da Conquista
-14.9027502,-41.9720654,Condeúba
-15.5329871,-40.9103203,Encruzilhada
-14.5287347,-40.3663895,Poções
-14.4280991,-41.4639925,Aracatu
-15.0338145,-41.2655571,Belo Campo
-15.5053557,-41.2352742,Cândido Sales
-14.8443374,-42.432116,Jacaraci
-14.662

-24.1415537,-52.7762208,Janiópolis
-23.5282979,-51.67107439999999,Mandaguari
-24.4379528,-51.9477938,Nova Tebas
-23.3954231,-52.6449469,São Manoel do Paraná
-25.0573431,-53.87961749999999,Vera Cruz do Oeste
-23.092495,-52.7900366,Amaporã
-23.2340437,-51.5853849,Pitangueiras
-23.7658108,-52.4466113,Terra Boa
-24.81621,-53.1192655,Braganey
-23.9918948,-51.4446034,Cruzmaltina
-23.1964442,-51.8245477,Iguaraçu
-23.470689,-52.5550034,Japurá
-23.6207804,-53.2065873,Maria Helena
-23.0224606,-52.9181483,Planaltina do Paraná
-25.3450718,-54.23861669999999,São Miguel do Iguaçu
-23.7309899,-53.4901742,Xambrê
-24.0702783,-53.8449535,Francisco Alves
-23.3190277,-51.5544529,Sabáudia
-24.6453413,-53.1337375,Anahy
-22.7893531,-51.7136586,Cafeara
-22.657229,-52.8611436,Diamante do Norte
-24.7168215,-53.0872038,Iguatu
-24.179855,-51.6905502,Jardim Alegre
-23.4856611,-51.7935562,Marialva
-22.7511155,-51.3805388,Porecatu
-23.8672702,-51.8570929,São Pedro do Ivaí
-24.1935281,-51.9236626,Godoy Moreira
-22.95

-3.4947131,-39.5803861,Itapipoca
-3.5414855,-40.454345,Meruoca
-3.918004,-38.603557,Pacatuba
-7.534395200000001,-39.11688609999999,Porteiras
-4.1431403,-40.5815662,Reriutaba
-3.5868372,-40.5454988,Alcântaras
-3.730929199999999,-40.9927503,Tianguá
-3.6210883,-39.50815120000001,Uruburetama
-4.3313286,-38.8789563,Baturité
-4.162546799999999,-40.9414761,Carnaubal
-4.0467078,-40.7526274,Graça
-4.2618162,-38.9330716,Guaramiranga
-4.3220187,-40.7092475,Ipu
-3.6844551,-39.58637239999999,Itapagé
-3.526268,-40.3421014,Massapê
-4.3037643,-38.9947042,Mulungu
-4.1477909,-38.84605639999999,Palmácia
-4.2238817,-38.7302215,Redenção
-4.0449071,-40.8651372,São Benedito
-17.7199562,-48.1591052,Ipameri
-18.1660958,-47.9444845,Catalão
-15.8041148,-46.92822899999999,Cabeceiras
-14.4710658,-48.4601862,Niquelândia
-15.4536994,-47.608917,Planaltina
-17.4465584,-51.1720161,Montividiu
-16.4854966,-48.6453981,Gameleira de Goiás
-16.9504597,-50.4466297,Paraúna
-16.4612113,-49.9615772,Anicuns
-17.3050562,-49.020528

,Região,Município,UF,Estado,location,"(Lat, Long, City)"
0,Sul de Minas,Aiuruoca,MG,Minas Gerais,"Aiuruoca, state of Minas Gerais, Brazil","-21.9750633,-44.6036109,Aiuruoca"
1,Sul de Minas,Tapiraí,MG,Minas Gerais,"Tapiraí, state of Minas Gerais, Brazil","-19.8868478,-46.0200063,Tapiraí"
2,Sul de Minas,Cabo Verde,MG,Minas Gerais,"Cabo Verde, state of Minas Gerais, Brazil","-21.4730317,-46.3970615,Cabo Verde"
3,Sul de Minas,Contagem,MG,Minas Gerais,"Contagem, state of Minas Gerais, Brazil","-19.9384589,-44.0518344,Contagem"
4,Sul de Minas,Estiva,MG,Minas Gerais,"Estiva, state of Minas Gerais, Brazil","-22.465012,-46.0157038,Estiva"
...,...,...,...,...,...,...
1416,Mato Grosso,Paranaíta,MT,Mato Grosso,"Paranaíta, state of Mato Grosso, Brazil","-9.6664274,-56.4734827,Paranaíta"
1417,Mato Grosso,Rondolândia,MT,Mato Grosso,"Rondolândia, state of Mato Grosso, Brazil","-10.8422109,-61.46081109999999,Rondolândia"
1418,Mato Grosso,Santo Antônio do Leverger,MT,Mato Grosso,"Santo Antônio do Leverger, state of Mato Gross...","-15.8637741,-56.0793735,Santo Antônio do Leverger"
1419,Mato Grosso,Tangará da Serra,MT,Mato Grosso,"Tangará da Serra, state of Mato Grosso, Brazil","-14.6197344,-57.4875157,Tangará da Serra"


In [6]:
df.drop(['location','Estado'], axis=1, inplace = True)
df.head()

,Região,Município,UF,"(Lat, Long, City)"
0,Sul de Minas,Aiuruoca,MG,"-21.9750633,-44.6036109,Aiuruoca"
1,Sul de Minas,Tapiraí,MG,"-19.8868478,-46.0200063,Tapiraí"
2,Sul de Minas,Cabo Verde,MG,"-21.4730317,-46.3970615,Cabo Verde"
3,Sul de Minas,Contagem,MG,"-19.9384589,-44.0518344,Contagem"
4,Sul de Minas,Estiva,MG,"-22.465012,-46.0157038,Estiva"


In [7]:
df[['Lat', 'Long', 'City']] = df['(Lat, Long, City)'].str.split(',', 2, expand=True)
df.drop(['(Lat, Long, City)'], axis=1, inplace = True)
df.head()

,Região,Município,UF,Lat,Long,City
0,Sul de Minas,Aiuruoca,MG,-21.9750633,-44.6036109,Aiuruoca
1,Sul de Minas,Tapiraí,MG,-19.8868478,-46.0200063,Tapiraí
2,Sul de Minas,Cabo Verde,MG,-21.4730317,-46.3970615,Cabo Verde
3,Sul de Minas,Contagem,MG,-19.9384589,-44.0518344,Contagem
4,Sul de Minas,Estiva,MG,-22.465012,-46.0157038,Estiva


In [8]:
df[df['Município']!=df['City']]

,Região,Município,UF,Lat,Long,City
206,Mantiqueira de Minas (Denominação de Origem),Brazópolis,MG,-22.473871,-45.6179988,Brasópolis
388,Chapada de Minas,Olhos-d`Água,MG,-17.4007437,-43.5723722,Olhos-d'Água
528,Montanhas de Minas,Pingo-d`Água,MG,-19.72694,-42.4087451,Pingo-d'Água
540,Montanhas de Minas,São José da Lapa,MG,-19.6986923,-43.9581514,Sao Jose da Lapa
638,Montanhas de Minas,Olaria,MG,-21.8600314,-43.9370888,"Olaria, Minas Gerais"
787,Média Mogiana,Mogi Mirim,SP,-22.4314429,-46.9569108,Moji-Mirim
795,Média Mogiana,Águas de Lindóia,SP,-22.4803601,-46.633882,Águas de Lindoia
827,Marília e Garça,Pompéia,SP,-22.1060594,-50.1736915,Pompeia
936,Oeste da Bahia (Indicação de Procedência),Luís Eduardo Magalhães,BA,-12.0806624,-45.7786578,Luis Eduardo Magalhães
953,Atlântico Baiano,Una,BA,-12.9777334,-38.501648,Salvador


In [9]:
df.drop(['City'], axis = 1, inplace  = True)

In [16]:
#correção manual

df.loc[df['Município'] == 'Una', 'Lat'] = '-15.295065865545306'
df.loc[df['Município'] == 'Una', 'Long'] = '-39.07529192851047'
df[df['Município'] == 'Una']

,Região,Município,UF,Lat,Long
953,Atlântico Baiano,Una,BA,-15.295065865545306,-39.07529192851047


In [17]:
df.to_csv('./Dados/cidades_regiao_com_gps.csv',index = False,encoding ='utf-8-sig')